<a href="https://colab.research.google.com/github/mattcat1221/Deep-Learning-Analysis/blob/main/AlphabetSoup_Charity_Optimization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Preprocessing

In [5]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import load_iris
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
import pandas as pd
import tensorflow as tf
import numpy as np
from tensorflow.keras.utils import to_categorical
import os
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import accuracy_score
from tensorflow.keras.callbacks import ReduceLROnPlateau

# Create a sequential model
nn = tf.keras.models.Sequential()


In [2]:
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [3]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df = application_df.drop(columns=['EIN'])

# Display the first few rows to verify the columns have been dropped
application_df.columns



Index(['NAME', 'APPLICATION_TYPE', 'AFFILIATION', 'CLASSIFICATION', 'USE_CASE',
       'ORGANIZATION', 'STATUS', 'INCOME_AMT', 'SPECIAL_CONSIDERATIONS',
       'ASK_AMT', 'IS_SUCCESSFUL'],
      dtype='object')

In [4]:
application_df.shape

(34299, 11)

In [5]:
# Determine the number of unique values in each column.
unique_values = application_df.nunique()

# Display the result
print(unique_values)


NAME                      19568
APPLICATION_TYPE             17
AFFILIATION                   6
CLASSIFICATION               71
USE_CASE                      5
ORGANIZATION                  4
STATUS                        2
INCOME_AMT                    9
SPECIAL_CONSIDERATIONS        2
ASK_AMT                    8747
IS_SUCCESSFUL                 2
dtype: int64


In [6]:
names_df = application_df.NAME.value_counts()

In [15]:
for name in names_df[names_df<25].index:
    application_df['NAME'] = application_df['NAME'].replace(name, 'Other')

In [17]:
print(application_df.nunique())

NAME                        96
APPLICATION_TYPE             9
AFFILIATION                  6
CLASSIFICATION               6
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64


In [18]:
# Get the value counts of the APPLICATION_TYPE column
application_type_counts = application_df['APPLICATION_TYPE'].value_counts()

# Display the value counts
print(application_type_counts)


APPLICATION_TYPE
T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: count, dtype: int64


In [19]:
# Determine which APPLICATION_TYPE values to replace
# Here, replace categories with fewer than 500 occurrences with "Other"
replace_application_types = application_type_counts[application_type_counts < 500].index

# Replace in DataFrame
application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(replace_application_types, 'Other')

# Display the updated value counts to verify
print(application_df['APPLICATION_TYPE'].value_counts())


APPLICATION_TYPE
T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: count, dtype: int64


In [20]:
# Look at CLASSIFICATION value counts to identify and replace with "Other"
classification_counts = application_df['CLASSIFICATION'].value_counts()

# Display the value counts
print(classification_counts)


CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: count, dtype: int64


In [21]:
# Get the value counts of the CLASSIFICATION column
classification_counts = application_df['CLASSIFICATION'].value_counts()

# Filter to display only those with counts greater than 1
classification_counts_filtered = classification_counts[classification_counts > 1]

# Display the filtered value counts
print(classification_counts_filtered)


CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: count, dtype: int64


In [22]:
# Get the value counts of the CLASSIFICATION column
classification_counts = application_df['CLASSIFICATION'].value_counts()

# Choose a cutoff value
cutoff_value = 1000

# Create a list of classifications to be replaced
classifications_to_replace = classification_counts[classification_counts < cutoff_value].index.tolist()

# Replace in DataFrame
application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(classifications_to_replace, 'Other')

# Display the updated value counts to verify
print(application_df['CLASSIFICATION'].value_counts())


CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: count, dtype: int64


In [23]:
# Convert categorical data to numeric with `pd.get_dummies`
application_df_encoded = pd.get_dummies(application_df, dtype="int")

# Display the first few rows of the new DataFrame to verify the conversion
application_df_encoded.head()

,STATUS,ASK_AMT,IS_SUCCESSFUL,NAME_AIR FORCE ASSOCIATION,NAME_ALABAMA FEDERATION OF WOMENS CLUBS,NAME_ALABAMA TREASURE FOREST ASSOCIATION,NAME_ALPHA PHI SIGMA,NAME_ALTRUSA INTERNATIONAL FOUNDATION INC,NAME_AMATEUR ATHLETIC UNION OF THE UNITED STATES,NAME_AMATEUR ATHLETIC UNION OF THE UNITED STATES INC,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,1,5000,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
1,1,108590,1,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,1,0
2,1,5000,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
3,1,6692,1,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,1,0
4,1,142590,1,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,1,0


In [24]:
# Assuming 'IS_SUCCESSFUL' is your target column
target = 'IS_SUCCESSFUL'

# Split the preprocessed data into features (X) and target (y) arrays
X = application_df_encoded.drop(columns=[target])
y = application_df_encoded[target]

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Display the shapes of the resulting datasets to verify
print(f"X_train shape: {X_train.shape}")
print(f"X_test shape: {X_test.shape}")
print(f"y_train shape: {y_train.shape}")
print(f"y_test shape: {y_test.shape}")

X_train shape: (27439, 139)
X_test shape: (6860, 139)
y_train shape: (27439,)
y_test shape: (6860,)


In [25]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [26]:
X_train = X_train.astype(int)
y_train = y_train.astype(int)

In [27]:
print(X_train.dtypes)
print(y_train.dtypes)




STATUS                                      int64
ASK_AMT                                     int64
NAME_AIR FORCE ASSOCIATION                  int64
NAME_ALABAMA FEDERATION OF WOMENS CLUBS     int64
NAME_ALABAMA TREASURE FOREST ASSOCIATION    int64
                                            ...  
INCOME_AMT_25000-99999                      int64
INCOME_AMT_50M+                             int64
INCOME_AMT_5M-10M                           int64
SPECIAL_CONSIDERATIONS_N                    int64
SPECIAL_CONSIDERATIONS_Y                    int64
Length: 139, dtype: object
int64


In [28]:
X_train = X_train.astype(np.float32)
y_train = y_train.astype(np.float32)
print(X_train.dtypes)
print(y_train.dtypes)



STATUS                                      float32
ASK_AMT                                     float32
NAME_AIR FORCE ASSOCIATION                  float32
NAME_ALABAMA FEDERATION OF WOMENS CLUBS     float32
NAME_ALABAMA TREASURE FOREST ASSOCIATION    float32
                                             ...   
INCOME_AMT_25000-99999                      float32
INCOME_AMT_50M+                             float32
INCOME_AMT_5M-10M                           float32
SPECIAL_CONSIDERATIONS_N                    float32
SPECIAL_CONSIDERATIONS_Y                    float32
Length: 139, dtype: object
float32


## Compile, Train and Evaluate the Model

In [29]:
# Define the number of input features
number_input_features = X_train.shape[1]

# Define the number of nodes in each hidden layer
hidden_nodes_layer1 = 80
hidden_nodes_layer2 = 30

# Initialize the Sequential model
nn = tf.keras.models.Sequential()

# Input layer
nn.add(tf.keras.layers.Input(shape=(number_input_features,)))

# First hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, activation='relu'))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation='relu'))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

# Check the structure of the model
nn.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense (Dense)                        │ (None, 80)                  │          11,200 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 30)                  │           2,430 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 1)                   │              31 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 13,661 (53.36 KB)

 Trainable params: 13,661 (53.36 KB)

 Non-trainable params: 0 (0.00 B)

In [30]:
# Define the number of input features
number_input_features = X_train.shape[1]

# Define the number of nodes in each hidden layer
hidden_nodes_layer1 = 80
hidden_nodes_layer2 = 30

# Initialize the Sequential model
nn = tf.keras.models.Sequential()

# Input layer
nn.add(tf.keras.layers.Input(shape=(number_input_features,)))

# First hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, activation='relu'))

# Add dropout after the first hidden layer
nn.add(tf.keras.layers.Dropout(0.2))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation='relu'))

# Add dropout after the second hidden layer (optional)
nn.add(tf.keras.layers.Dropout(0.2))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

# Check the structure of the model
nn.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense_3 (Dense)                      │ (None, 80)                  │          11,200 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 80)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_4 (Dense)                      │ (None, 30)                  │           2,430 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ (None, 30)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_5 (Dense)                      │ (None, 1)                   │              31 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 13,661 (53.36 KB)

 Trainable params: 13,661 (53.36 KB)

 Non-trainable params: 0 (0.00 B)

In [31]:
# Compile the model
nn.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


In [32]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

215/215 - 1s - 5ms/step - accuracy: 0.6026 - loss: 0.7404
Loss: 0.74042147397995, Accuracy: 0.6026238799095154


In [33]:
nn.save('AlphabetSoupCharity_Optimization.h5')

In [34]:
# Train the model
fit_model = nn.fit(X_train_scaled, y_train, epochs=100,batch_size=32,validation_data=(X_test, y_test))

# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")



Epoch 1/100
858/858 ━━━━━━━━━━━━━━━━━━━━ 5s 4ms/step - accuracy: 0.7125 - loss: 0.5618 - val_accuracy: 0.4659 - val_loss: 10146.6064
Epoch 2/100
858/858 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.7695 - loss: 0.4730 - val_accuracy: 0.4659 - val_loss: 12605.2598
Epoch 3/100
858/858 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.7736 - loss: 0.4632 - val_accuracy: 0.4659 - val_loss: 14416.0078
Epoch 4/100
858/858 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.7699 - loss: 0.4704 - val_accuracy: 0.4659 - val_loss: 17754.1836
Epoch 5/100
858/858 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.7695 - loss: 0.4642 - val_accuracy: 0.4659 - val_loss: 11922.7793
Epoch 6/100
858/858 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - accuracy: 0.7778 - loss: 0.4591 - val_accuracy: 0.4659 - val_loss: 5794.4727
Epoch 7/100
858/858 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.7751 - loss: 0.4637 - val_accuracy: 0.4659 - val_loss: 7676.6270
Epoch 8/100
858/858 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.7736

In [35]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

215/215 - 1s - 3ms/step - accuracy: 0.7631 - loss: 0.4897
Loss: 0.4897252023220062, Accuracy: 0.7631195187568665


In [8]:
# Save the entire model to a HDF5 file
nn.save('AlphabetSoupCharity.keras')

In [9]:
nn.save('/content/my_trained_model.h5')

In [10]:
# Save the model using the .keras format
nn.save('AlphabetSoupCharity_Optimization.keras')

print("Model saved as 'AlphabetSoupCharity_Optimization.keras'")


Model saved as 'AlphabetSoupCharity_Optimization.keras'


In [11]:
nn.save('AlphabetSoupCharity_Optimization.h5')